# Build Python Models to Predict Probability of Call Center Escalation
- XGBoost:  Serialize with Pickle and ONNX
- Tensorflow: Serialize with H5

In [1]:
import csv
import pandas as pd
import numpy as np
import os, shutil # used to create necessary folders
import pickle
import json
import sklearn
import tensorflow as tf
from xgboost import XGBClassifier 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
 
    
# from sasctl import Session, register_model
# from sasctl.services import model_management as mm
# from sasctl.services import model_repository as mr


 


2022-06-20 15:05:46.299914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-20 15:05:46.299944: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
callcenter = pd.read_csv('Data_orig/call_center_v2.csv')

callcenter.dtypes



CALL_CENTER_ID                int64
ESCALATED                     int64
DAILY_CALLS                   int64
MONTHLY_CALLS               float64
CALL_TARGET                 float64
REASON                       object
MONTHS_CUSTOMER             float64
NEGATIVE_RATING             float64
DAYS_PROD_OUT_OF_SERVICE    float64
PRODUCT_AGE                 float64
CALL_INQUIRIES              float64
NUM_CALLERS                 float64
CALL_TIME                   float64
PRODUCT_TYPE                 object
FEEDBACK                     object
Sentiment                     int64
dtype: object

In [3]:
#callcenter['MONTHS_CUSTOMER'] = callcenter['MONTHS_CUSTOMER'].astype(float)
#df['DataFrame Column'] = df['DataFrame Column'].astype(float)
#gene.dtypes
callcenter.dtypes

#callcenter.head()
 

CALL_CENTER_ID                int64
ESCALATED                     int64
DAILY_CALLS                   int64
MONTHLY_CALLS               float64
CALL_TARGET                 float64
REASON                       object
MONTHS_CUSTOMER             float64
NEGATIVE_RATING             float64
DAYS_PROD_OUT_OF_SERVICE    float64
PRODUCT_AGE                 float64
CALL_INQUIRIES              float64
NUM_CALLERS                 float64
CALL_TIME                   float64
PRODUCT_TYPE                 object
FEEDBACK                     object
Sentiment                     int64
dtype: object

In [4]:
target         = ['ESCALATED']
numeric_inputs = ['DAILY_CALLS', 'MONTHLY_CALLS', 'CALL_TARGET', 'MONTHS_CUSTOMER', 
                  'NEGATIVE_RATING', 'DAYS_PROD_OUT_OF_SERVICE', 'PRODUCT_AGE', 'CALL_INQUIRIES','NUM_CALLERS','CALL_TIME']
class_inputs   = ['REASON', 'PRODUCT_TYPE']
 
impute_values = callcenter[numeric_inputs].mean()
pickle.dump(impute_values, open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle','wb'))
   
    
callcenter           =callcenter.fillna(impute_values)
callcenter.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
callcenter.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)

 
x              = callcenter.drop(['ESCALATED', 'CALL_CENTER_ID', 'FEEDBACK', 'Sentiment'], axis=1)
y              = callcenter[target].astype("category")

 

In [5]:
callcenter2 =callcenter.head()
callcenter2.drop(['ESCALATED', 'FEEDBACK', 'Sentiment'], axis=1, inplace=True)
callcenter2.to_csv('Data_orig/callcenter2_ds2_test.csv', index=False)
 
    

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Split Dataset Into Training and Test Set

In [6]:
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

print("train size: " + str(x_train.shape))
print("test size: " + str(x_test.shape))
print("train sample")

 
 
 
 

train size: (4172, 12)
test size: (1788, 12)
train sample


# Custom preprocessing for categorical inputs to use in XGB model

In [7]:
 
ohe = OneHotEncoder(sparse = False, handle_unknown="ignore")
enc = ohe.fit_transform(x_train[class_inputs])
enc2 = ohe.transform(x_test[class_inputs])

 
x_train[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc, index=x_train.index)
x_train.drop(class_inputs,1,inplace=True)

x_test[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc2, index=x_test.index)
x_test.drop(class_inputs,1,inplace=True)


print(x_train.shape)
print(x_test.shape)


 
 


(4172, 18)
(1788, 18)


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/3776711327.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train.drop(class_inputs,1,inplace=True)
/tmp/ipykernel_30983/3776711327.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_test.drop(class_inputs,1,inplace=True)


In [8]:
#Pickle the one hot encoding
pickle.dump(ohe, open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle', 'wb'))

# Normalize Inputs with Standard Scaling

In [9]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#scaler = MinMaxScaler()
# fit scaler on the training dataset
#scaler.fit(X_train)
#X_train_scaled = scaler.transform(X_train)
pickle.dump(scaler, open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'wb'))

type(x_train)

numpy.ndarray

# Train the XGBoost Model

In [10]:
# Create XGBoost classifer object

 
# x_train_array = x_train.values
# y_train_array = y_train.values
    
booster = xgb.XGBClassifier(max_depth=3,
                        booster='dart',
                        eta=0.3,
                        silent=1,
                        n_estimators=100)


# Train XGBoost Classifer
booster.fit(x_train,y_train)

modelName = "CALLCENTER_XGBoost"
 

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

[15:05:51] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[15:05:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Train the Tensorflow Model

In [11]:
x_train.shape

(4172, 18)

In [12]:
# Train Tensorflow Classified with Kera API 
clf_tens = tf.keras.models.Sequential([tf.keras.layers.Dense(units=32, activation = 'relu',input_shape=(x_train.shape[-1],)),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=16, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=8, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                               
                                 # use sigmoid activation for binary target, softmax did not work
                                 tf.keras.layers.Dense(1, activation = 'sigmoid')])

# use binary_crossentropy loss function for binary target
clf_tens.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0006),
              loss='binary_crossentropy',
              metrics = ['acc'])

# train a model
clf_tens.fit(x_train,y_train, epochs=500,batch_size=int(len(x_train)/10),verbose=1,validation_data=(x_test, y_test))

modelName2 = "CALLCENTER_Tensorflow"

2022-06-20 15:05:52.268965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-20 15:05:52.269002: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-20 15:05:52.269027: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eeclxvm068.exnet.sas.com): /proc/driver/nvidia/version does not exist
2022-06-20 15:05:52.269379: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/keras/optimizer_v2/rmsp

Epoch 1/500
11/11 [==============================] - 1s 19ms/step - loss: 2.5152 - acc: 0.7675 - val_loss: 0.7813 - val_acc: 0.3289
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 2.1225 - acc: 0.7744 - val_loss: 0.7677 - val_acc: 0.3540
Epoch 3/500
11/11 [==============================] - 0s 4ms/step - loss: 1.9490 - acc: 0.7663 - val_loss: 0.7560 - val_acc: 0.3842
Epoch 4/500
11/11 [==============================] - 0s 4ms/step - loss: 1.8884 - acc: 0.7709 - val_loss: 0.7435 - val_acc: 0.4167
Epoch 5/500
11/11 [==============================] - 0s 4ms/step - loss: 1.9413 - acc: 0.7737 - val_loss: 0.7323 - val_acc: 0.4457
Epoch 6/500
11/11 [==============================] - 0s 4ms/step - loss: 1.6107 - acc: 0.7768 - val_loss: 0.7226 - val_acc: 0.4760
Epoch 7/500
11/11 [==============================] - 0s 4ms/step - loss: 1.8124 - acc: 0.7721 - val_loss: 0.7140 - val_acc: 0.5006
Epoch 8/500
11/11 [==============================] - 0s 4ms/step - loss: 1.6016 - 

11/11 [==============================] - 0s 4ms/step - loss: 0.6809 - acc: 0.7960 - val_loss: 0.5798 - val_acc: 0.8009
Epoch 64/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6344 - acc: 0.7967 - val_loss: 0.5786 - val_acc: 0.8009
Epoch 65/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6759 - acc: 0.7958 - val_loss: 0.5770 - val_acc: 0.8009
Epoch 66/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6318 - acc: 0.7965 - val_loss: 0.5758 - val_acc: 0.8009
Epoch 67/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6304 - acc: 0.7972 - val_loss: 0.5749 - val_acc: 0.8009
Epoch 68/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6564 - acc: 0.7967 - val_loss: 0.5737 - val_acc: 0.8009
Epoch 69/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6290 - acc: 0.7979 - val_loss: 0.5730 - val_acc: 0.8009
Epoch 70/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6325 - acc: 0

Epoch 125/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5357 - acc: 0.8008 - val_loss: 0.5270 - val_acc: 0.8009
Epoch 126/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5415 - acc: 0.8003 - val_loss: 0.5269 - val_acc: 0.8009
Epoch 127/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5470 - acc: 0.8001 - val_loss: 0.5268 - val_acc: 0.8009
Epoch 128/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5356 - acc: 0.7999 - val_loss: 0.5261 - val_acc: 0.8009
Epoch 129/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5380 - acc: 0.7996 - val_loss: 0.5257 - val_acc: 0.8009
Epoch 130/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5388 - acc: 0.7999 - val_loss: 0.5251 - val_acc: 0.8009
Epoch 131/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5369 - acc: 0.7999 - val_loss: 0.5244 - val_acc: 0.8009
Epoch 132/500
11/11 [==============================] - 0s 5ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.5115 - acc: 0.7999 - val_loss: 0.5080 - val_acc: 0.8009
Epoch 187/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5159 - acc: 0.8001 - val_loss: 0.5077 - val_acc: 0.8009
Epoch 188/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5111 - acc: 0.8001 - val_loss: 0.5074 - val_acc: 0.8009
Epoch 189/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5094 - acc: 0.7999 - val_loss: 0.5070 - val_acc: 0.8009
Epoch 190/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5141 - acc: 0.8001 - val_loss: 0.5069 - val_acc: 0.8009
Epoch 191/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5152 - acc: 0.8003 - val_loss: 0.5066 - val_acc: 0.8009
Epoch 192/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5092 - acc: 0.8003 - val_loss: 0.5065 - val_acc: 0.8009
Epoch 193/500
11/11 [==============================] - 0s 6ms/step - loss: 0.5101 -

Epoch 248/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5012 - acc: 0.8003 - val_loss: 0.4998 - val_acc: 0.8009
Epoch 249/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5035 - acc: 0.8003 - val_loss: 0.4998 - val_acc: 0.8009
Epoch 250/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5121 - acc: 0.8003 - val_loss: 0.4996 - val_acc: 0.8009
Epoch 251/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5041 - acc: 0.8003 - val_loss: 0.5002 - val_acc: 0.8009
Epoch 252/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5050 - acc: 0.7999 - val_loss: 0.5003 - val_acc: 0.8009
Epoch 253/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5071 - acc: 0.8006 - val_loss: 0.5004 - val_acc: 0.8009
Epoch 254/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5034 - acc: 0.8003 - val_loss: 0.5003 - val_acc: 0.8009
Epoch 255/500
11/11 [==============================] - 0s 4ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.5001 - acc: 0.8003 - val_loss: 0.4984 - val_acc: 0.8009
Epoch 310/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5006 - acc: 0.8003 - val_loss: 0.4986 - val_acc: 0.8009
Epoch 311/500
11/11 [==============================] - 0s 5ms/step - loss: 0.5027 - acc: 0.8003 - val_loss: 0.4982 - val_acc: 0.8009
Epoch 312/500
11/11 [==============================] - 0s 8ms/step - loss: 0.4987 - acc: 0.8003 - val_loss: 0.4980 - val_acc: 0.8009
Epoch 313/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5026 - acc: 0.8003 - val_loss: 0.4983 - val_acc: 0.8009
Epoch 314/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5018 - acc: 0.8003 - val_loss: 0.4983 - val_acc: 0.8009
Epoch 315/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5005 - acc: 0.8003 - val_loss: 0.4982 - val_acc: 0.8009
Epoch 316/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5001 -

Epoch 371/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5002 - acc: 0.8003 - val_loss: 0.4964 - val_acc: 0.8009
Epoch 372/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4979 - acc: 0.8003 - val_loss: 0.4959 - val_acc: 0.8009
Epoch 373/500
11/11 [==============================] - 0s 6ms/step - loss: 0.4970 - acc: 0.8006 - val_loss: 0.4958 - val_acc: 0.8009
Epoch 374/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4990 - acc: 0.8003 - val_loss: 0.4955 - val_acc: 0.8009
Epoch 375/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5004 - acc: 0.8003 - val_loss: 0.4964 - val_acc: 0.8009
Epoch 376/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5000 - acc: 0.8003 - val_loss: 0.4961 - val_acc: 0.8009
Epoch 377/500
11/11 [==============================] - 0s 6ms/step - loss: 0.4979 - acc: 0.8003 - val_loss: 0.4956 - val_acc: 0.8009
Epoch 378/500
11/11 [==============================] - 0s 6ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.4965 - acc: 0.8003 - val_loss: 0.4939 - val_acc: 0.8009
Epoch 433/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4988 - acc: 0.8003 - val_loss: 0.4942 - val_acc: 0.8009
Epoch 434/500
11/11 [==============================] - 0s 6ms/step - loss: 0.4988 - acc: 0.8003 - val_loss: 0.4942 - val_acc: 0.8009
Epoch 435/500
11/11 [==============================] - 0s 6ms/step - loss: 0.4972 - acc: 0.8003 - val_loss: 0.4937 - val_acc: 0.8009
Epoch 436/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4989 - acc: 0.8006 - val_loss: 0.4940 - val_acc: 0.8009
Epoch 437/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4998 - acc: 0.8001 - val_loss: 0.4942 - val_acc: 0.8009
Epoch 438/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4970 - acc: 0.8003 - val_loss: 0.4932 - val_acc: 0.8009
Epoch 439/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5022 -

Epoch 494/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4988 - acc: 0.8003 - val_loss: 0.4929 - val_acc: 0.8009
Epoch 495/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4959 - acc: 0.8003 - val_loss: 0.4932 - val_acc: 0.8009
Epoch 496/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4988 - acc: 0.8003 - val_loss: 0.4933 - val_acc: 0.8009
Epoch 497/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4984 - acc: 0.8003 - val_loss: 0.4941 - val_acc: 0.8009
Epoch 498/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4960 - acc: 0.8003 - val_loss: 0.4910 - val_acc: 0.8009
Epoch 499/500
11/11 [==============================] - 0s 5ms/step - loss: 0.4969 - acc: 0.8003 - val_loss: 0.4909 - val_acc: 0.8009
Epoch 500/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5003 - acc: 0.8003 - val_loss: 0.4916 - val_acc: 0.8009


# Pickle the trained XGB Model and H5 for Tensorflow Model

In [13]:
#pickle.dump(model, open(project_dir+"/HMEQ_XGB.pickle", 'wb'))
pickle.dump(booster, open("/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName + ".pickle", 'wb'))

clf_tens.save(filepath = "/sasinside/userdata/gegrab/resources/hmeq" + '/' + modelName2 + '.h5')

#tf.keras.models.save_model("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_Tensorflow.h5")

# Write the XGBoost Model to ONNX


In [14]:
model     = booster
modelName3 = "CALLCENTER_ONNX_XGBoost"

column_count = x_train.shape[1] #second position
column_count



18

In [15]:
import onnx
from onnxmltools.convert.common import data_types

initial_type = [('float_input', data_types.FloatTensorType([1, column_count]))]

In [16]:
from onnxmltools.convert import convert_xgboost

booster_onnx = convert_xgboost(booster, initial_types=initial_type)
onnx.save(booster_onnx, "/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName3 +".onnx")
 

# Write Out Parameters for Copying

In [17]:
# Define input variables and perform formatting to match with score code file

input_params = ''
for col in callcenter.columns:
    input_params += col
    if col != callcenter.columns[-1]:
        input_params += ', '

input_cols = ''
for col in callcenter.columns:
    input_cols += "\"" + col + "\""
    if col != callcenter.columns[-1]:
        input_cols += ', '


print(input_cols)
print("")
print(input_params)

"CALL_CENTER_ID", "ESCALATED", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE", "FEEDBACK", "Sentiment"

CALL_CENTER_ID, ESCALATED, DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE, FEEDBACK, Sentiment


# Test ONNX Scoring Code

In [18]:
def scoreONNX(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE):
    "Output: P_ESCALATED"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
     
    
    

    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_ESCALATED = float(pred_onnx[0][0])
    
    
     
    return (P_ESCALATED)


 

In [19]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/callcenter2_ds2_test.csv') 
X_test2 
 
# gene2 = gene.drop(['BAD','Loan_ID'], axis=1)
 
for i in range(5):
    print(scoreONNX(**X_test2.iloc[i]))

0.9617191553115845
0.9451096653938293
0.9710361957550049
0.9905694723129272
0.9195560812950134


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1737868913.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1737868913.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-

# Write Out ONNX Score Code

In [20]:
score_code = """
def scoreONNX(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE ):
    "Output: P_ESCALATED"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
     
    
    

    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_ESCALATED = float(pred_onnx[0][0])
    
    
     
    return (P_ESCALATED)"""



f = open('Data_orig/CALLCENTER_ONNX_XGB.py',"w+")
f.write(score_code)
f.close()

# Test XGBoost Score Code

In [21]:
def scoreXGB(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE ):
    "Output: P_ESCALATED"
        
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_XGBoost.pickle", 'rb'))

    P_ESCALATED = float(sess.predict_proba(test_data)[0,0])
    
     
     
    return (P_ESCALATED)



In [22]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/callcenter2_ds2_test.csv') 
X_test2 
 
 
 
for i in range(5):
    print(scoreXGB(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1216535939.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.9617192149162292
0.9451096653938293


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1216535939.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1216535939.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-

0.9710361361503601
0.9905694127082825
0.9195561408996582


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/1216535939.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


# Write Out XGBoost Score Code

In [23]:
score_code = """
def scoreXGB(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE ):
    "Output: P_ESCALATED"
        
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_XGBoost.pickle", 'rb'))

    P_ESCALATED = float(sess.predict_proba(test_data)[0,0])
    
     
     
    return (P_ESCALATED)"""

f = open('Data_orig/CALLCENTER_XGB.py',"w+")
f.write(score_code)
f.close()

# Test Tensorflow Score Code

In [24]:
def scoreTF(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE ):
    "Output: P_ESCALATED"
        
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
    
    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
     
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_Tensorflow.h5")
    
    P_ESCALATED = float(sess.predict(test_data))
    P_ESCALATED = float(tf.keras.activations.sigmoid(P_ESCALATED))
     
    
    return (P_ESCALATED)



In [25]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/callcenter2_ds2_test.csv') 
X_test2 
  
 
 
for i in range(5):
    print(scoreTF(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/715628238.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.5482414364814758
0.5504912734031677


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/715628238.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/715628238.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.5441893935203552


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/715628238.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.5335507988929749
0.5505303740501404


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30983/715628238.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


# Write Out Tensorflow Score Code

In [26]:
score_code = """
def scoreTF(CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE ):
    "Output: P_ESCALATED"
        
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[CALL_CENTER_ID,DAILY_CALLS, MONTHLY_CALLS, CALL_TARGET, REASON, MONTHS_CUSTOMER, NEGATIVE_RATING, DAYS_PROD_OUT_OF_SERVICE, PRODUCT_AGE, CALL_INQUIRIES, NUM_CALLERS, CALL_TIME, PRODUCT_TYPE]],
                              columns = ["CALL_CENTER_ID", "DAILY_CALLS", "MONTHLY_CALLS", "CALL_TARGET", "REASON", "MONTHS_CUSTOMER", "NEGATIVE_RATING", "DAYS_PROD_OUT_OF_SERVICE", "PRODUCT_AGE", "CALL_INQUIRIES", "NUM_CALLERS", "CALL_TIME", "PRODUCT_TYPE"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'ProdFailure',regex = True, inplace=True)
    inputArray.PRODUCT_TYPE.replace(np.nan,'Other',regex = True, inplace=True)
    
    
    inputArray.drop(['CALL_CENTER_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/callcenter_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/callcenter_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
     
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/CALLCENTER_Tensorflow.h5")
    
    P_ESCALATED = float(sess.predict(test_data))
    P_ESCALATED = float(tf.keras.activations.sigmoid(P_ESCALATED))
     
    
    return (P_ESCALATED)"""


f = open('Data_orig/CALLCENTER_Tensorflow.py',"w+")
f.write(score_code)
f.close()




In [27]:
# prefixModelFile = 'PYTHON CALL CENTER'
# project_name_swat = 'CALL CENTER PYTHON SWAT'

# model_name_swat = prefixModelFile + ' SWAT GBOOST'

# import os
# os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'

# import swat

# # Connect to the CAS server
 
# conn=swat.CAS("eeclxvm068", 5570, 'gegrab','Demopw123')

# #hmeq = pd.read_csv('./hmeq_combined.csv', sep =',' )

# # Upload the training data to CAS
# #tbl = conn.upload('Data_orig/hmeq_combined.csv').casTable
# tbl = conn.upload('Data_orig/call_center_v2.csv').casTable 

# features = list(tbl.columns)
# print(features)
# #print(features[:-1])
# print(features[2:])


# conn.loadactionset('decisionTree')
# tbl.decisiontree.gbtreetrain(target='ESCALATED',
#                              inputs= features[2:14],
#                              savestate = dict(name = 'callcenter_gtb_model_astore', caslib = 'gegrab_p', replace = True))

# astore = conn.CASTable('callcenter_gtb_model_astore', caslib = 'gegrab_p', replace=True)


In [28]:
# with Session('eeclxvm067', 'gegrab', 'Demopw123', verify_ssl=False):
#     model = register_model(astore, 'Call Center Python SWAT GBoost', 'Call Center Python SWAT ', force=True)